# **Create Computational Meshes**
<hr>

In [1]:
from vmtk import vmtkscripts
import os

The default current directory when loading a notebook from GDrive is the home folder of the current user. To load the user's libraries and modules, first change to the file location using the os' python module, and load the necessary modules, then change to the former directory. 

In [2]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

In [3]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       =  '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/'

In [4]:
# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/


In [5]:
# Get array with pacthes info
surface = vmtk_functions.readSurface(meshesDir+'wallSurfaceMesh.stl')

Reading STL surface file.


In [6]:
vmtk_functions.viewSurface(surface)

Quit renderer


If surface have its dimension sin meters, then scale it back to millimeters here.

In [ ]:
surfaceScaling = vmtkscripts.vmtkSurfaceScaling()
surfaceScaling.Surface = surface
surfaceScaling.ScaleFactor = 1000.0
surfaceScaling.Execute()

In [ ]:
surface = surfaceScaling.Surface

In [7]:
capsGeoInfo = vmtk_functions.getPatchInfo(surface)
print(capsGeoInfo)


CellEntityId: 2
  Origin: -0.000026, 0.000220, 0.000000
  Normal: 0.000000, 0.000000, -1.000000
  Radius: 1.437221

CellEntityId: 3
  Origin: 11.155675, -12.383680, 3.246148
  Normal: 0.833069, -0.457098, 0.311542
  Radius: 1.128742

CellEntityId: 4
  Origin: 1.269730, -9.533875, 14.482897
  Normal: 0.058478, -0.393267, 0.917563
  Radius: 0.949900

Quit renderer
wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
BoundaryNormals
BoundaryRadius
Point1Array
Point2Array
CellEntityIds
converting cell connectivity list
[ ((-2.5754096e-05, 0.00021974262, 1.1313995e-08), (2.0047723027890876e-08, 9.9977738696114211e-09, -0.99999999999999978),  1.43722061, 2, 'inlet')
 ((11.155675, -12.38368, 3.2461481), (0.83306856218849057, -0.45709793008602928, 0.31154173557369258),  1.12874189, 3, 'outlet')
 ((1.2697303, -9.5338745, 14.482897), (0.058478362385568444, -0.3932665183644341, 0.91756292790533911),  0.94990039, 4, 'outlet')]


<hr/>
# **Meshing the Aneurysm Surface with VMTK**

Finally, we use the final generated surface obtained in the previous section to generate a CFD tetrahedral mesh. It is possible in VMTK to generate a boudary layer mesh with radius adaptative cell refinement.

For the radius adaptative procedure, we need to first calculate the centerlines, and then to generate a DistanceToCenterlines fields defined on the surface. It is importante to note that the radius adaptative mesh, in my opinion, is not quite good when meshing aneurysm geometries, due to poor quality elements inside the aneurysm volume (you will need to generate a centerline inside the aneurysm dome, however, depending on the aneurysm size, this won't be very helpful).

In [ ]:
# Computing the centerlines of the surface
# for generating a radius adaptative mesh
centerlines = vmtkscripts.vmtkCenterlines()
centerlines.Surface = surface
# centerlines.SeedSelectorName = 'openprofiles'
centerlines.AppendEndPoints = 1
centerlines.Execute()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
# Calculating distance to centerlines
distanceToCenterlines = vmtkscripts.vmtkDistanceToCenterlines()

distanceToCenterlines.Surface = centerlines.Surface
distanceToCenterlines.Centerlines = centerlines.Centerlines
distanceToCenterlines.UseRadiusInformation = 1
distanceToCenterlines.RadiusArrayName = centerlines.RadiusArrayName
distanceToCenterlines.PrintInputMembers()
distanceToCenterlines.PrintOutputMembers()
distanceToCenterlines.Execute()

# viewSurface(distanceToCenterlines.Surface)

In [ ]:
aneurysmResolutionArray = vmtkscripts.vmtkSurfaceRegionDrawing()

aneurysmResolutionArray.Surface = surface
# aneurysmResolutionArray.SurfaceInputFileName = surfacesDir+'surfaceWithThickness.vtp'
# aneurysmResolutionArray.IORead()

# General options
# aneurysmResolutionArray.Binary = 1
aneurysmResolutionArray.InsideValue = 0.05
aneurysmResolutionArray.OutsideValue = 0.1
aneurysmResolutionArray.ContourScalarsArrayName = 'ResolutionArray'

aneurysmResolutionArray.Execute()

# Debug
aneurysmResolutionArray.PrintInputMembers()
aneurysmResolutionArray.PrintOutputMembers()

aneurysmArraySmoothing = vmtkscripts.vmtkSurfaceArraySmoothing()

aneurysmArraySmoothing.Surface = aneurysmResolutionArray.Surface

aneurysmArraySmoothing.SurfaceArrayName = aneurysmResolutionArray.ContourScalarsArrayName
aneurysmArraySmoothing.Connexity = 1
aneurysmArraySmoothing.Relaxation = 1.0
aneurysmArraySmoothing.Iterations = 30
# aneurysmArraySmoothing.SurfaceOutputFileName = surfacesDir+'surfaceToWallMesh.vtp'
# aneurysmArraySmoothing.PrintInputMembers()
# aneurysmArraySmoothing.PrintOutputMembers()
aneurysmArraySmoothing.SurfaceArrayName = aneurysmResolutionArray.ContourScalarsArrayName
aneurysmArraySmoothing.Execute()
# aneurysmArraySmoothing.IOWrite()

# Debug
# aneurysmArraySmoothing.PrintInputMembers()
# aneurysmArraySmoothing.PrintOutputMembers()

# Remeshing the surface with quality triangles
surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

surfaceRemesh.Surface = aneurysmArraySmoothing.Surface #resolutionArraySmoothing.Surface
# surfaceRemesh.SurfaceInputFileName = surfacesDir+'surfaceWithThickness.vtp'
# surfaceRemesh.IORead()

# surfaceRemesh.ElementSizeMode = "edgelength"
# surfaceRemesh.TargetEdgeLength = 0.1

# Options: 'area', 'edgelength', 'areaarray', 'edgelengtharray'
surfaceRemesh.ElementSizeMode = 'edgelengtharray'
surfaceRemesh.TargetEdgeLengthArrayName = resolutionArrayCreator.ContourScalarsArrayName
surfaceRemesh.TargetEdgeLengthFactor = 1

# If ElementSizeMode = "area"
# surfaceRemesh.TargetArea = 0.1

# If ElementSizeMode = 'areaarray'
# surfaceRemesh.ElementSizeMode = 'areaarray'
# surfaceRemesh.TargetAreaArrayName = 'SurfaceResolutionArray'
# surfaceRemesh.TargetAreaFactor = 0.01

# surfaceRemesh.SurfaceOutputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.vtp'

# Debug
# surfaceRemesh.PrintInputMembers()
# surfaceRemesh.PrintOutputMembers()

surfaceRemesh.Execute()
surfaceRemesh.IOWrite()

In [ ]:
vmtk_functions.viewSurface(surfaceRemesh.Surface)

In [ ]:
surfaceRemesh.SurfaceOutputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.vtp'
surfaceRemesh.IOWrite()

In [ ]:
fieldProjector = vmtkscripts.vmtkSurfaceProjection()

# fieldProjector.SurfaceInputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl'
# fieldProjector.IORead()

fieldProjector.Surface = surfaceRemesh.Surface
fieldProjector.ReferenceSurface = aneurysmArraySmoothing.Surface
# fieldProjector.ReferenceSurfaceInputFileName = surfacesDir+'surfaceWithThickness.vtp'
# fieldProjector.IORead()

# fieldProjector.SurfaceOutputFileName = surfacesDir+'surfaceToWallMesh.vtp'
fieldProjector.Execute()
# fieldProjector.IOWrite()

# Debug
fieldProjector.PrintInputMembers()
fieldProjector.PrintOutputMembers()

In [ ]:
vmtk_functions.viewSurface(surface)

In [ ]:
# Generating a radius adaptative mesh
mesh = vmtkscripts.vmtkMeshGenerator()

# Needed parameters for uniform mesh
mesh.Surface = surface #fieldProjector.Surface
mesh.ElementSizeMode = "edgelength"
# mesh.TargetEdgeLengthArrayName = distanceToCenterlines.DistanceToCenterlinesArrayName #aneurysmResolutionArray.ContourScalarsArrayName 
mesh.TargetEdgeLength = 0.2
mesh.TargetEdgeLengthFactor = 1.0
# mesh.MeshOutputFileName = meshesDir+'meshWithBL.vtu'

# mesh.SkipRemeshing = 1
# mesh.CappingMethod = 'simple'
# mesh.SkipCapping = 0

mesh.TriangleSplitFactor = 3
mesh.VolumeElementScaleFactor = 0.8

# To create boundary layer refinement
mesh.BoundaryLayer = 1
mesh.NumberOfSubLayers = 8
mesh.SubLayerRatio = 0.8
mesh.BoundaryLayerThicknessFactor = 0.4 
mesh.BoundaryLayerOnCaps = 0

# Parameters for radius adaptative mesh
# mesh.TargetEdgeLengthArrayName = distanceToCenterlines.DistanceToCenterlinesArrayName
# mesh.TargetEdgeLengthFactor = 0.3
    
mesh.PrintInputMembers()
mesh.PrintOutputMembers()
mesh.Execute()

In [ ]:
mesh.MeshOutputFileName = meshesDir+'meshWithBL.vtu'
mesh.RemeshedSurfaceOutputFileName = meshesDir+'wallSurfaceMesh.vtp'
mesh.IOWrite()

In [ ]:
meshWriter = vmtkscripts.vmtkMeshWriter()

meshWriter.Mesh = mesh.Mesh
# meshWriter.CellEntityIdsArrayName = wallMesh.CellEntityIdsArrayName
meshWriter.Mode = 'ascii'
meshWriter.OutputFileName = meshesDir+'meshWithBL.vtk'
# meshWriter.WriteVTKMeshFile()
meshWriter.WriteRegionMarkers = 1

meshWriter.Execute()
meshWriter.IOWrite()
# Debug
meshWriter.PrintInputMembers()
meshWriter.PrintOutputMembers()

<hr> 
# **Generating a Mesh for the Aneurysm and Tissue Wall**

For fluid-solid interaction simulations, a mesh for the solid wall -- artery and aneurysm wall -- must be created in the context of partitioned FSI methods. While the operations above handle the creation of the fluid domain mesh -- or it can be created in an external mesh generator, such as *snappyHexMesh*--, the operations below use vmtk scripts to generate a prismatic layer mesh for the solid domain. Investigation of a variable wall thickness are performed here, also using the aforementioned vmtk scripts.

The literature reports that the aneurysm and its surrounding arteries have different thickness: studies performed by Costalat et al. (2011), Robertson et al. (2015) and Cebral et al. (2015) show that the aneurysm thickness ranges from 0.1 to 0.6 mm, with mean thickness around 0.3 mm, whereas the intracranial arteries have mean thickness with 0.6+-0.12 mm and 0.51+-0.08 mm in the basilar and middle cerebral arteries, respectively, for example. Of course, these measures change from patient to patient and nothing assures that the thickness of our aneurysms cases are the same. The study by Nakagawa et al. (2016) performed measurements of the so-called *wall-to-lumen ratio (WLR)*, which is given by:

\begin{equation}
    WLR = \frac{\text{artery wall thickness}}{\text{artery lumen diameter}}
\end{equation}

in patients being operated: during the procedure, they measured the artery outer diameter in different positions of healthy intracranial arteries, while the inner diameter was measured by *indocyanine green angiography*, a technique used for flow measurements during the operation. The authors claim that their study was the first to directly measure this parameter in human -- former studies have obtained it for rats and human cadavers. 

Their results show that the mean WLR for the following arteries are:

    Medium-sized intracranial arteries (2-3mm): 0.070+-0,01
    Large-size intracranial arteries (> 3mm): 0.088+-0.012
    
In the code below, I will use this parameter to calculate the parent and daughter arteries wall thickness based on their diameter. 

Regarding the aneurysm wall thickness, by this time (07/13/2018), I did not found any study reporting the WLR for intracranial aneurysms, or any measured related to it for this kind of pathology. Since it is well reported that the wall of intracranial aneurysms have a thickness distribution, probably the WLR definition as given above is not applicable to aneurysms, at least not in a straightforward manner. Therefore, we adopt the following rule-- of course, this can be changed if any study reporting the WLR, or other related measure, be released in the future: once we calculate the surrounding arteries wall thickness, we choose a slighlty smaller thickness for the aneurysm dome region that is in the range presented above.   

In the code below, I attempt to create a *thickness array* defined on a surface: the thickness should be smaller on the aneurysm region than on the arteries region, according to the results show above. I will create the thickness array with different techiniques, for example: using the *vmtkdistancetocenterlines* script to create an array that have increasing values along the aneurysm mean line from the artery centerline. In this case, we don't have too much control on the values of the array. The other technique involves the *vmtkssurfaceresolution*, *vmtkgeodesicsurfaceresolution*, *vmtkdistancetospheres* or *vmtksurfaceregiondrawing* to create an array based on the Eulerian or geodesic distance to seeds positioned on the surface. In this case, we have more control on the array bounding values. 

Together with the *ThicknessArray*, we need to create a *SurfaceRefinementArray*, which will be used with the *vmtksurfaceremeshing* script to generate a surface with different refinement levels on the surface triangles. This procedure is needed to refine close to high curvature regions of the arteries and aneurysm surface, usually close to the aneurysm neck and on arteries curves. 

<hr>
The distance to spheres script in the next cell will be used to create the *SurfaceRefinementArray*, by positioning spheres close to the aneurysm neck.

The input surface here is the remeshed surface when it was initially extracted, because is a high quality surface. 
The output surface will be named as 'surfaceToWallMesh.vtp' which will be used when the external layer will be created.

In the next cell, we define two variables holding the aneurysm and arteries mean thickness that will be used below, in millimeters. 

In [8]:
# Definition of medium and large arteries
largeArteryDiameter = 3.00 # mm

# First we define the WLR:
wlrMedium = 0.07
wlrLarge = 0.088

# Then the arteries diameter
# - For terminal cases, for normal extraction cases,
#   there will be a parent artery and 2 daughter arteries 
#   after the bifurcations
daughtersDiameter = []

for artery in range(len(capsGeoInfo)): 
    # The inlet radius is attributed to the parent artery diameter
    if capsGeoInfo['PatchType'][artery] == 'inlet':
        parentArteryDiameter = round(2 * capsGeoInfo['Radius'][artery], 2) # mm
        
    # The other two are stored in a list with the daughter 
    # arteries diameters
    else:
        daughtersDiameter.append(round(2 * capsGeoInfo['Radius'][artery], 2))

# # Allocate variables for wall thickness
# parentArteryThickness = 0.0 
# daughter1ArteryThickness = 0.0
# daughter2ArteryThickness = 0.0

# Defines dictionary with arteries diameters 
# and wall thickness in tuple
arteriesInfo = {'parentArtery': {'diameter': parentArteryDiameter, 'thickness': None},
                'daughter1Artery': {'diameter': daughtersDiameter[0], 'thickness': None},
                'daughter2Artery': {'diameter': daughtersDiameter[1], 'thickness': None}}

# Calculate artery wall thickness 
# based on size of the artery
for artery in arteriesInfo.keys():
    # Compare artery diameter with large artery
    if arteriesInfo[artery]['diameter'] > largeArteryDiameter:
        arteriesInfo[artery]['thickness'] = round(wlrLarge * arteriesInfo[artery]['diameter'], 2)
    else:
        arteriesInfo[artery]['thickness'] = round(wlrMedium * arteriesInfo[artery]['diameter'], 2)

arteriesInfo

{'parentArtery': {'diameter': 2.8700000000000001,
  'thickness': 0.20000000000000001},
 'daughter1Artery': {'diameter': 2.2599999999999998, 'thickness': 0.16},
 'daughter2Artery': {'diameter': 1.8999999999999999, 'thickness': 0.13}}

In [9]:
# Define here the aneurysm wall thickness
aneurysmThickness = 0.5*arteriesInfo['parentArtery']['thickness'] # mm
aneurysmThickness

0.10000000000000001

# **Generating the Thickness array**

The overall procedure of creating the thickness array is decomposed in two steps: first a *base array* must be created and then an *auxiliar array*. Operations with these two are then carried out to generate the final thickness array.

## Generating the base array

Using *vmtkdistancetospheres*, by placing a sphere on the aneurysm dome center and delimiting the aneurysm neck with the sphere surface by increasing its diameter. The script, then, generates an array on the surface with values ranging from *minDistance* to *maxDistance* to the sphere surface, corresponding to the *Euclidean* distance in this case.
Setup:

    minDistance = minimum thickness (aneurysm)
    maxDistance = maximum thickness (surrounding arteries)
    
The *minDistance* will correspond to the aneurysm thickness for that case whereas the *maxDistance* to the arteries thickness. The neck, therefore, will have a spatial gradient of values ranging from *minDistance* to *maxDistance* along the neck lenght.

In [ ]:
# Gnerating the array with the aneurysm thickness value
thicknessArrayCreator = vmtkscripts.vmtkSurfaceRegionDrawing()
thicknessArrayCreator.Surface = surface
thicknessArrayCreator.OutsideValue = aneurysmThickness
thicknessArrayCreator.InsideValue = aneurysmThickness
thicknessArrayCreator.ContourScalarsArrayName = 'ThicknessArray'
thicknessArrayCreator.Binary = 1
thicknessArrayCreator.Execute()

surface = thicknessArrayCreator.Surface

# View surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = thicknessArrayCreator.Surface
surfaceViewer.ArrayName = thicknessArrayCreator.ContourScalarsArrayName
surfaceViewer.Legend = 1
surfaceViewer.Execute()

In [ ]:
thicknessArrayCreator.SurfaceOutputFileName = surfacesDir+'test.vtp'
thicknessArrayCreator.IOWrite()

I need to understand what is exactly the rule used to define the inside and outside regions. It seems always when I isolate the parent artery, the inside region does not include the artery itself, but the rest of the surface; and when I paint the daughter arteries, the artery is the inside zone and the rest if the ouside region. But I DONT KNOW WHY! There no logic on this reasoning, it is just by observation. I am still waiting for someone to answer my question on VMTK users group.

In [ ]:
# Now we loop over arteries with different thicknesses
for artery in arteriesInfo.keys():
    print('Isolate the artery '+artery+' by drawing it with a contour.')
    
    drawArtery = vmtkscripts.vmtkSurfaceRegionDrawing()
    drawArtery.Surface = surfaceNormals.Surface
    
    if artery == 'parentArtery':
        drawArtery.InsideValue = 0.0
        drawArtery.OutsideValue = arteriesInfo[artery]['thickness'] - aneurysmThickness
    else:
        drawArtery.InsideValue = arteriesInfo[artery]['thickness'] - aneurysmThickness   
        drawArtery.OutsideValue = 0.0
        
    drawArtery.Binary = 1
    drawArtery.Execute()
    
    # Now add the array in the ThicknessArray of the original surface
    arrayOperator = vmtkscripts.vmtkSurfaceArrayOperation()
    arrayOperator.Surface = surface
    arrayOperator.Surface2 = drawArtery.Surface
    arrayOperator.InputArrayName = thicknessArrayCreator.ContourScalarsArrayName
    arrayOperator.Input2ArrayName = drawArtery.ContourScalarsArrayName
    arrayOperator.Operation = 'add'
    arrayOperator.ResultArrayName = thicknessArrayCreator.ContourScalarsArrayName
    arrayOperator.Execute()
    
    # View surface
    surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
    surfaceViewer.Surface = arrayOperator.Surface
    surfaceViewer.ArrayName = arrayOperator.ResultArrayName
    surfaceViewer.Legend = 1
    surfaceViewer.Execute()
    
    surface = arrayOperator.Surface
    

In [ ]:
# Since the array created by RegionDrawing is discontinous
# we use the array smoothing
thicknessArraySmoother = vmtkscripts.vmtkSurfaceArraySmoothing()
thicknessArraySmoother.Surface = surface
thicknessArraySmoother.SurfaceArrayName = thicknessArrayCreator.ContourScalarsArrayName
# General options
thicknessArraySmoother.Connexity = 1
thicknessArraySmoother.Relaxation = 1.0
thicknessArraySmoother.Iterations = 30
thicknessArraySmoother.Execute()

# Debug
# thicknessArraySmoother.PrintInputMembers()
# thicknessArraySmoother.PrintOutputMembers()

In [ ]:
thicknessArraySmoother.SurfaceOutputFileName = surfacesDir+'surfaceWithThickness.vtp'
thicknessArraySmoother.IOWrite()

In [ ]:
# View surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = thicknessArraySmoother.Surface
surfaceViewer.ArrayName = thicknessArraySmoother.SurfaceArrayName
surfaceViewer.Legend = 1
surfaceViewer.Execute()

In [ ]:
baseArrayCreator = vmtkscripts.vmtkDistanceToSpheres()

baseArrayCreator.Surface = surfaceNormals.Surface #surfaceRemesh.Surface

baseArrayCreator.DistanceToSpheresArrayName = 'BaseArray'
# Set the DistanceOffset of the field creator to the 
# minimum distance == aneurysmThickness
baseArrayCreator.DistanceOffset = aneurysmThickness
baseArrayCreator.Scale = 0.15

## If DistanceOffset attribut is 0.0
## then this value is the array value 
## closest to the spheres
baseArrayCreator.MinDistance = aneurysmThickness
baseArrayCreator.MaxDistance = arteryThickness

baseArrayCreator.Execute()

# Debug
# baseArrayCreator.PrintInputMembers()
# baseArrayCreator.PrintOutputMembers()

## Generating auxiliar array for subtraction

To avoid bad mesh elements near high curvature regions (near the neck and arteries curves) the base array generated above is modified by *decreasing* the thickness value on those regions. This can be accomplished by creating an array with values defined on these high curvature/neck regions, which we will call *AuxiliarArray*, with this value being defined as **how much we want to diminish the thickness**. 

The *AuxiliarArray* is created with the script *vmtksurfaceregiondrawing* which allows us to define closed regions on the surface which will have a specified value inside, the value to be subtracted from the *BaseArray*. The regions specified by the user must be the ones where we want to change the base array.

After that, this auxiliar array must be smoothed, using the *vmtkarraysmoothing* script.

In [ ]:
auxiliarArrayCreator = vmtkscripts.vmtkSurfaceRegionDrawing()

auxiliarArrayCreator.Surface = baseArrayCreator.Surface

# General options
# regionDrawing.Binary = 1
auxiliarArrayCreator.InsideValue = 0.15
auxiliarArrayCreator.OutsideValue = 0.0
auxiliarArrayCreator.ContourScalarsArrayName = 'AuxiliarArray'

auxiliarArrayCreator.Execute()

# Debug
auxiliarArrayCreator.PrintInputMembers()
auxiliarArrayCreator.PrintOutputMembers()

# Since the array created by RegionDrawing is discontinous
# we use the array smoothing
auxiliarArraySmoother = vmtkscripts.vmtkSurfaceArraySmoothing()

auxiliarArraySmoother.Surface = auxiliarArrayCreator.Surface
auxiliarArraySmoother.SurfaceArrayName = auxiliarArrayCreator.ContourScalarsArrayName

# General options
auxiliarArraySmoother.Connexity = 1
auxiliarArraySmoother.Relaxation = 1.0
auxiliarArraySmoother.Iterations = 15

auxiliarArraySmoother.Execute()

# Debug
auxiliarArraySmoother.PrintInputMembers()
auxiliarArraySmoother.PrintOutputMembers()

## Generating Final Thickness array

The final *ThicknessArray* is cretaed by subtracting *AuxiliarArray* of *BaseArray* with the *vmtksurfacearrayoperation* filter.

In [ ]:
thicknessArrayCreator = vmtkscripts.vmtkSurfaceArrayOperation()

thicknessArrayCreator.Surface = auxiliarArraySmoother.Surface
thicknessArrayCreator.Surface2 = auxiliarArraySmoother.Surface

thicknessArrayCreator.Operation = 'subtract'
thicknessArrayCreator.InputArrayName = baseArrayCreator.DistanceToSpheresArrayName
thicknessArrayCreator.Input2ArrayName = auxiliarArrayCreator.ContourScalarsArrayName

thicknessArrayCreator.ResultArrayName = 'ThicknessArray'
thicknessArrayCreator.SurfaceOutputFileName = surfacesDir+'surfaceWithThickness.vtp'

thicknessArrayCreator.Execute()
thicknessArrayCreator.IOWrite()

# Debug
# thicknessArrayCreator.PrintInputMembers()
# thicknessArrayCreator.PrintOutputMembers()

Thickness array creation finished on the initial surface. Filesaved as 'surfaceWithThickness.vtp' (need to be .vtp to store arrays).
Proceed with generation of resolution array for remeshing.

## Computing surface outwards normals

The surface normals will be used at the final process of creating the wall mesh. To avoid incorrect orientation of the final artery wall surface, the normals are flipped here, in this way the final surface will have the correct orientation regarding its volume. 

In [14]:
# Building normals to surface array on remeshed surface for thicknesse creation
surfaceNormals = vmtkscripts.vmtkSurfaceNormals()

# Loading surface
surfaceNormals.Surface = surface #fieldProjector.Surface
# surfaceNormals.SurfaceInputFileName = surfacesDir+'patches/wall.stl'
# surfaceNormals.IORead()

# General options
# surfaceNormals.ComputeCellNormals = 1
surfaceNormals.FlipNormals = 1

surfaceNormals.Execute()

## **Generating resolution array for remeshing**

Now we generate a remeshed surface with refined high quality triangles in the regions of high curvature/aneurysm neck to avoid bad volume elements whn extruding the surface to generate the wall mesh. We can use here also the script *vmtkdistancetospheres* by placing spheres on the regions that we want to refine. The *minDistance* and *maxDistance* correponding in this case to the minimum edgelenght (or element area) and maximum edgelenght that we wish to use on the remeshign procedure. Note that these parameters can be scaled if needed with the *edgelenghtfactor* parameter of *vmtksurfaceremeshing*.

If you think that it is not necessary to refine specific regions of the surface -- depending on the aneurysm and arteries geometries, it can be the case --, then change the boolean flag *refinement* in the cell below to 'False' -- note that the default behavior is 'True' --  and skip the next cell until the final wall mesh creation cell.

In [13]:
refinement = True

In [ ]:
resolutionArrayCreator = vmtkscripts.vmtkSurfaceRegionDrawing()

# resolutionArrayCreator.Surface = thicknessArraySmoother.Surface
resolutionArrayCreator.SurfaceInputFileName = surfacesDir+'patches/wall.stl' #'surfaceWithThickness.vtp'
resolutionArrayCreator.IORead()

# General options
# regionDrawing.Binary = 1
resolutionArrayCreator.InsideValue = 0.05
resolutionArrayCreator.OutsideValue = 0.1
resolutionArrayCreator.ContourScalarsArrayName = 'ResolutionArray'

resolutionArrayCreator.Execute()

# Debug
# resolutionArrayCreator.PrintInputMembers()
# resolutionArrayCreator.PrintOutputMembers()

In [ ]:
resolutionArraySmoothing = vmtkscripts.vmtkSurfaceArraySmoothing()

resolutionArraySmoothing.Surface = resolutionArrayCreator.Surface

resolutionArraySmoothing.SurfaceArrayName = resolutionArrayCreator.ContourScalarsArrayName
resolutionArraySmoothing.Connexity = 1
resolutionArraySmoothing.Relaxation = 1.0
resolutionArraySmoothing.Iterations = 20
# resolutionArraySmoothing.SurfaceOutputFileName = surfacesDir+'surfaceToWallMesh.vtp'
# resolutionArraySmoothing.PrintInputMembers()
# resolutionArraySmoothing.PrintOutputMembers()
resolutionArraySmoothing.SurfaceArrayName = resolutionArrayCreator.ContourScalarsArrayName

resolutionArraySmoothing.Execute()
# resolutionArraySmoothing.IOWrite()

# Debug
# resolutionArrayCreator.PrintInputMembers()
# resolutionArrayCreator.PrintOutputMembers()

In [ ]:
# Remeshing the surface with quality triangles
surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

surfaceRemesh.Surface = resolutionArraySmoothing.Surface #resolutionArraySmoothing.Surface
# surfaceRemesh.SurfaceInputFileName = surfacesDir+'surfaceWithThickness.vtp'
# surfaceRemesh.IORead()

# surfaceRemesh.ElementSizeMode = "edgelength"
# surfaceRemesh.TargetEdgeLength = 0.1

# Options: 'area', 'edgelength', 'areaarray', 'edgelengtharray'
surfaceRemesh.ElementSizeMode = 'edgelengtharray'
surfaceRemesh.TargetEdgeLengthArrayName = resolutionArrayCreator.ContourScalarsArrayName
surfaceRemesh.TargetEdgeLengthFactor = 1

# If ElementSizeMode = "area"
# surfaceRemesh.TargetArea = 0.1

# If ElementSizeMode = 'areaarray'
# surfaceRemesh.ElementSizeMode = 'areaarray'
# surfaceRemesh.TargetAreaArrayName = 'SurfaceResolutionArray'
# surfaceRemesh.TargetAreaFactor = 0.01

# surfaceRemesh.SurfaceOutputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.vtp'

# Debug
# surfaceRemesh.PrintInputMembers()
# surfaceRemesh.PrintOutputMembers()

surfaceRemesh.Execute()
surfaceRemesh.IOWrite()

In [ ]:
surfaceRemesh.SurfaceOutputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl'
surfaceRemesh.IOWrite()

In [ ]:
surfaceSmoothed = vmtk_functions.smoothSurface(surfaceRemesh.Surface)
vmtk_functions.viewSurface(surfaceSmoothed)

In [ ]:
vmtk_functions.writeSurface(surfaceSmoothed,surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl','ascii')

## **Projecting Normals and Thickness arrays on new remeshed surfaces**

In [10]:
fieldProjector = vmtkscripts.vmtkSurfaceProjection()

# fieldProjector.SurfaceInputFileName = surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl'
# fieldProjector.IORead()

fieldProjector.Surface = surface #vmtk_functions.readSurface(surfacesDir+'surfaceWithThickness.vtp')

# fieldProjector.ReferenceSurface = thicknessArrayCreator.Surface
fieldProjector.ReferenceSurfaceInputFileName = surfacesDir+'surfaceWithThickness.vtp'
fieldProjector.IORead()

fieldProjector.SurfaceOutputFileName = surfacesDir+'surfaceToWallMesh.vtp'
fieldProjector.Execute()
fieldProjector.IOWrite()

# Debug
fieldProjector.PrintInputMembers()
fieldProjector.PrintOutputMembers()

Reading VTK XML surface file.
Computing projection
Writing VTK XML surface file.
Input vmtksurfaceprojection members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    ReferenceSurface = vtkPolyData
    ReferenceSurfaceInputFileName = /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/surfaceWithThickness.vtp
    SurfaceOutputFileName = /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/surfaceToWallMesh.vtp
Output vmtksurfaceprojection members:
    Id = 0
    Surface = vtkPolyData


In [11]:
# View surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = fieldProjector.Surface
surfaceViewer.ArrayName = 'ThicknessArray'
surfaceViewer.Legend = 1
surfaceViewer.Execute()

Quit renderer


In [15]:
# Conveting surface to mesh with normals and curvature informations
surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()

# surfaceToMesh.SurfaceInputFileName = surfacesDir+'surfaceToWallMesh.vtp'
# surfaceToMesh.IORead()
# Needs to be a .vtp surface file!! not stl
if refinement:
    surfaceToMesh.Surface = surfaceNormals.Surface #fieldProjector.Surface
else:
    surfaceToMesh.Surface = thicknessArraySmoother.Surface

# Debug
# surfaceToMesh.PrintInputMembers()
# surfaceToMesh.PrintOutputMembers()

surfaceToMesh.Execute()
# surfaceToMesh.IOWrite()

# Creating aneurysm and arteries external wall
wallMesh = vmtkscripts.vmtkBoundaryLayer()

wallMesh.Mesh = surfaceToMesh.Mesh
wallMesh.WarpVectorsArrayName = 'Normals'#surfaceNormals.NormalsArrayName
wallMesh.ThicknessArrayName = "ThicknessArray" #thicknessArrayCreator.ResultArrayName
wallMesh.MeshOutputFileName = meshesDir+'meshWall.vtu'
wallMesh.ThicknessRatio = 1

# wallMesh.UseWarpVectorMagnitudeAsThickness = 1
# wallMesh.Thickness = 0.2
wallMesh.NumberOfSubLayers = 5
# wallMesh.ConstantThickness = 1

# Setup
wallMesh.NumberOfSubsteps = 7000
wallMesh.Relaxation = 0.01
wallMesh.LocalCorrectionFactor = 0.5

# Entity ids for new mesh
wallMesh.VolumeCellEntityId = 0
wallMesh.InnerSurfaceCellEntityId = 1
wallMesh.OuterSurfaceCellEntityId = 2
wallMesh.SidewallCellEntityId = 3

wallMesh.IncludeSidewallCells = 0
wallMesh.IncludeSurfaceCells = 0
wallMesh.NegateWarpVectors = 1

wallMesh.PrintInputMembers()
wallMesh.PrintOutputMembers()

wallMesh.Execute()
wallMesh.IOWrite()

Input vmtkboundarylayer members:
    Id = 0
    Disabled = 0
    Mesh = vtkUnstructuredGrid
    MeshInputFileName = 
    WarpVectorsArrayName = Normals
    ThicknessArrayName = ThicknessArray
    CellEntityIdsArrayName = CellEntityIds
    Thickness = 1.0
    ThicknessRatio = 1
    MaximumThickness = 10000000000.0
    NumberOfSubLayers = 5
    NumberOfSubsteps = 7000
    Relaxation = 0.01
    LocalCorrectionFactor = 0.5
    InnerSurfaceCellEntityId = 1
    OuterSurfaceCellEntityId = 2
    SidewallCellEntityId = 3
    VolumeCellEntityId = 0
    SubLayerRatio = 1.0
    UseWarpVectorMagnitudeAsThickness = 0
    ConstantThickness = 0
    IncludeSurfaceCells = 0
    IncludeSidewallCells = 0
    NegateWarpVectors = 1
    MeshOutputFileName = /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/meshes/meshWall.vtu
    InnerSurfaceMeshOutputFileName = 
Output vmtkboundarylayer members:
    Id = 0
    Mesh = vtkUnstructuredGrid
    InnerSurfaceMe

In [17]:
meshWriter = vmtkscripts.vmtkMeshWriter()

meshWriter.Mesh = wallMesh.Mesh
# meshWriter.CellEntityIdsArrayName = wallMesh.CellEntityIdsArrayName
meshWriter.Mode = 'ascii'
meshWriter.OutputFileName = meshesDir+'meshWall.vtk'
# meshWriter.WriteVTKMeshFile()
meshWriter.WriteRegionMarkers = 1

meshWriter.Execute()
meshWriter.IOWrite()
# Debug
meshWriter.PrintInputMembers()
meshWriter.PrintOutputMembers()

Writing VTK mesh file.
Input vmtkmeshwriter members:
    Id = 0
    Disabled = 0
    Mesh = vtkUnstructuredGrid
    MeshInputFileName = 
    Format = vtk
    GuessFormat = 1
    Compressed = 1
    OutputFileName = /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/meshes/meshWall.vtk
    Mesh = vtkUnstructuredGrid
    Mode = ascii
    CellEntityIdsArrayName = CellEntityIds
    CellEntityIdsOffset = -1
    WriteRegionMarkers = 1
Output vmtkmeshwriter members:
    Id = 0


In [ ]:
# # Surface curvature array
# surfaceCurvature = vmtkscripts.vmtkSurfaceCurvature()

# surfaceCurvature.Surface = surfaceNormals.Surface
# surfaceCurvature.CurvatureType = 'minimum'
# surfaceCurvature.AbsoluteCurvature = 1
# surfaceCurvature.MedianFiltering = 1
# surfaceCurvature.BoundedReciprocal = 1
# surfaceCurvature.Epsilon = 2
# # surfaceCurvature.CurvatureOnBoundaries = 1
# # surfaceCurvature.Offset = -0.1
# # surfaceCurvature.SurfaceOutputFileName = casePath+"surfaces/surfaceCurvature.vtp"

# surfaceCurvature.PrintInputMembers()
# # surfaceCurvature.PrintOutputMembers()

# surfaceCurvature.Execute()
# # surfaceCurvature.IOWrite()
# surfaceCurvature.OutputText('Surface curvature computed.')

# # View surface
# surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
# surfaceViewer.Surface = surfaceCurvature.Surface
# surfaceViewer.ArrayName = 'Curvature'
# surfaceViewer.Legend = 1
# # surfaceViewer.ScalarRange = [0,1]
# surfaceViewer.Execute()

In [ ]:
# # 'vmtkurfaceresolution' uses the Euclidean distance to the RBF interpolation
# # whereas 'vmtkgeodesicsurfaceresolution' uses the geodesic distance along the surface

# surfaceResolution = vmtkscripts.vmtkSurfaceResolution()
    
# surfaceResolution.SurfaceInputFileName = surfacesDir+'patches/wall.stl'
# surfaceResolution.IORead()

# surfaceResolution.ResolutionArrayName = 'ResolutionArray'
# # RBF Options: 'thinplatespline', 'biharmonic', 'triharmonic'
# surfaceResolution.RBFType = 'biharmonic'

# surfaceResolution.Execute()
# surfaceResolution.PrintInputMembers()
# surfaceResolution.PrintOutputMembers()

# surfaceResolution.SurfaceOutputFileName = surfacesDir+'surfaceToWallMesh.vtp'
# surfaceResolution.IOWrite()

# 'vmtkurfaceresolution' uses the Euclidean distance to the RBF interpolation
# whereas 'vmtkgeodesicsurfaceresolution' uses the geodesic distance along the surface

In [ ]:
# surfaceGeodesicResolution = vmtkscripts.vmtkGeodesicSurfaceResolution()

# surfaceGeodesicResolution.SurfaceInputFileName = surfacesDir+'patches/wall.stl'
# surfaceGeodesicResolution.IORead()

# surfaceGeodesicResolution.ResolutionArrayName = 'ResolutionArray'
# # RBF Options: 'thinplatespline', 'biharmonic', 'triharmonic'
# surfaceGeodesicResolution.RBFType = 'biharmonic'

# surfaceGeodesicResolution.Execute()
# surfaceGeodesicResolution.PrintInputMembers()
# surfaceGeodesicResolution.PrintOutputMembers()

# surfaceGeodesicResolution.SurfaceOutputFileName = surfacesDir+'surfaceGeodesicResolution.vtp'
# surfaceGeodesicResolution.IOWrite()